In [ ]:
# pip install optuna

In [ ]:
# pip install datasets

In [ ]:
import optuna
import torch
from torch import nn
from torch.utils.data import DataLoader, IterableDataset
from transformers import AutoImageProcessor, AutoModelForImageClassification
from sklearn.metrics import average_precision_score, precision_recall_fscore_support
from tqdm import tqdm
from torchvision import transforms
import numpy as np
from PIL import Image

In [ ]:
# Custom Model with Dropout
class CustomDINOv2WithDropout(nn.Module):
    def __init__(self, base_model, num_labels, dropout_rate=0.3):
        super(CustomDINOv2WithDropout, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.base_model.config.hidden_size, num_labels)

    def forward(self, pixel_values):
        outputs = self.base_model(pixel_values=pixel_values, output_hidden_states=False)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits


# Multi-label Metrics
def multi_label_metrics(logits, y_true, labels, threshold=0.5):
    probs = torch.sigmoid(logits).cpu().numpy()
    y_pred = probs > threshold

    # Compute overall metrics
    mean_prec, mean_rec, mean_f1, _ = precision_recall_fscore_support(
        y_true=y_true, y_pred=y_pred, average="weighted", zero_division=np.nan
    )
    mean_ap = average_precision_score(y_true, probs, average="weighted")

    # Compute label-wise metrics
    precs, recs, f1s, _ = precision_recall_fscore_support(
        y_true=y_true, y_pred=y_pred, average=None, zero_division=np.nan
    )
    aps = average_precision_score(y_true, probs, average=None)

    # Combine metrics into a dictionary
    metrics = {
        "mean_ap": mean_ap,
        "mean_precision": mean_prec,
        "mean_recall": mean_rec,
        "mean_f1": mean_f1,
        "label_aps": {labels[i]: aps[i] for i in range(len(labels))},
        "label_f1s": {labels[i]: f1s[i] for i in range(len(labels))}
    }
    return metrics


# Dataset class
class StreamDataset(IterableDataset):
    def __init__(self, dataset, split_name, label_keys, image_transforms):
        self.dataset = dataset
        self.split_name = split_name
        self.label_keys = label_keys
        self.image_transforms = image_transforms

    def process_item(self, item):
        image = item["image"]
        labels = [int(item[key]) for key in self.label_keys]
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        processed_image = self.image_transforms(image)
        processed_labels = torch.tensor(labels, dtype=torch.float32)
        return processed_image, processed_labels

    def __iter__(self):
        for item in self.dataset[self.split_name]:
            yield self.process_item(item)


# Dataset processing
def process_dataset(
    model, dataset, split_name, label_keys, image_transforms, optimizer=None, train=False, batch_size=8
):
    model.train() if train else model.eval()
    running_loss = 0.0
    all_logits, all_labels = [], []

    processed_dataset = StreamDataset(dataset, split_name, label_keys, image_transforms)
    loader = DataLoader(processed_dataset, batch_size=batch_size, collate_fn=lambda x: tuple(zip(*x)))

    batch_count = 0
    for batch in tqdm(loader, desc="Training" if train else "Validation"):
        batch_count += 1
        batch_images, batch_labels = map(torch.stack, batch)
        batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

        if train:
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                logits = model(batch_images)
                loss = torch.nn.BCEWithLogitsLoss()(logits, batch_labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()
        else:
            with torch.no_grad():
                logits = model(batch_images)
                loss = torch.nn.BCEWithLogitsLoss()(logits, batch_labels)
                running_loss += loss.item()

        all_logits.append(logits.detach().cpu())
        all_labels.append(batch_labels.detach().cpu())

    all_logits = torch.cat(all_logits, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    return running_loss / batch_count, all_logits, all_labels


# Objective Function for Optuna
def objective(trial):
    global model, processor, device

    # Hyperparameters to tune
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16)

    # Redefine the model with tuned dropout rate
    model = CustomDINOv2WithDropout(base_model.base_model, num_labels=len(label_keys), dropout_rate=dropout_rate).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scaler = torch.cuda.amp.GradScaler()

    # Path to save the best model for this trial
    trial_model_path = f"best_model_trial_{trial.number}.pth"

    # Training Loop
    num_epochs =5  # Shorter for tuning
    patience, no_improvement = 3, 0
    best_val_map = 0.0
    accumulation_steps = max(1, 64 // batch_size)  # Simulate effective batch size of 64

    for epoch in range(num_epochs):
        print(f"Trial {trial.number}, Epoch {epoch + 1}/{num_epochs}")

        # Training
        model.train()
        train_dataset = StreamDataset(ds, "train", label_keys, image_transforms)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=lambda x: tuple(zip(*x)))

        train_loss = 0.0
        optimizer.zero_grad()
        for i, (batch_images, batch_labels) in enumerate(tqdm(train_loader, desc="Training")):
            batch_images, batch_labels = torch.stack(batch_images).to(device), torch.stack(batch_labels).to(device)

            with torch.cuda.amp.autocast():
                logits = model(batch_images)
                loss = nn.BCEWithLogitsLoss()(logits, batch_labels)
                loss = loss / accumulation_steps  # Normalize for gradient accumulation

            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0 or (i + 1) == batch_size * accumulation_steps:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            train_loss += loss.item() * accumulation_steps  # De-normalize for logging

        # Validation
        val_loss, val_logits, val_labels = process_dataset(
            model, ds, "validation", label_keys, image_transforms, batch_size=batch_size
        )
        val_metrics = multi_label_metrics(val_logits, val_labels.numpy(), label_keys)
        print(f"Validation Metrics (Trial {trial.number}, Epoch {epoch + 1}): {val_metrics}")

        # Log mean_ap for the trial
        trial.report(val_metrics["mean_ap"], epoch)
        print(f"Trial {trial.number}, Epoch {epoch + 1}: mean_ap = {val_metrics['mean_ap']:.4f}")

        # Save the best model of the trial
        if val_metrics["mean_ap"] > best_val_map:
            best_val_map = val_metrics["mean_ap"]
            no_improvement = 0
            torch.save(model.state_dict(), trial_model_path)
            print(f"Best model saved for trial {trial.number} with mAP: {best_val_map:.4f}")
        else:
            no_improvement += 1

        # Prune trial if mean_ap is not improving
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        # Early stopping
        if no_improvement >= patience:
            print(f"Trial {trial.number}: Early stopping triggered.")
            break

        torch.cuda.empty_cache()  # Clear memory to prevent fragmentation

    return best_val_map

# Load dataset
from datasets import load_dataset
ds = load_dataset("MITLL/LADI-v2-dataset", streaming=True)

# Define label keys
label_keys = [
    'bridges_any', 'buildings_any', 'buildings_affected_or_greater',
    'buildings_minor_or_greater', 'debris_any', 'flooding_any',
    'flooding_structures', 'roads_any', 'roads_damage', 'trees_any',
    'trees_damage', 'water_any'
]

# Load DINOv2 model
model_name = "facebook/dinov2-base"
processor = AutoImageProcessor.from_pretrained(model_name)
base_model = AutoModelForImageClassification.from_pretrained(
    model_name, ignore_mismatched_sizes=True
)

# Define transformations
image_transforms = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Device and scaler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.cuda.amp.GradScaler()

# Run Optuna Study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=2)

# Print best hyperparameters
print("Best hyperparameters:", study.best_params)


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-0abc238c160f>:222: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
[I 2024-12-05 16:41:12,475] A new study created in memory with name: no-name-12853c83-60cb-4795-98bb-f0774facc515
<ipython-input-7-0abc238c160f>:121: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Trial 0, Epoch 1/5


...

Validation Metrics (Trial 1, Epoch 3): {'mean_ap': 0.9410091707370805, 'mean_precision': 0.8937241511097336, 'mean_recall': 0.8966855777844133, 'mean_f1': 0.8911220877952073, 'label_aps': {'bridges_any': 0.7831663969092, 'buildings_any': 0.9916689983151098, 'buildings_affected_or_greater': 0.8687148975245242, 'buildings_minor_or_greater': 0.7527988899094158, 'debris_any': 0.7752574350884568, 'flooding_any': 0.8710479675052797, 'flooding_structures': 0.8309885766868328, 'roads_any': 0.9891705843020282, 'roads_damage': 0.4592882295490415, 'trees_any': 0.9959456596407743, 'trees_damage': 0.8368729658378681, 'water_any': 0.9647280853567688}, 'label_f1s': {'bridges_any': 0.7083333333333334, 'buildings_any': 0.9577735124760077, 'buildings_affected_or_greater': 0.7522388059701492, 'buildings_minor_or_greater': 0.6105263157894737, 'debris_any': 0.6918238993710691, 'flooding_any': 0.774869109947644, 'flooding_structures': 0.8305084745762712, 'roads_any': 0.9464882943143813, 'roads_damage': 0.4,

Training: 0it [00:00, ?it/s]<ipython-input-7-0abc238c160f>:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 502it [25:16,  3.02s/it]
Validation: 56it [03:03,  3.27s/it]


Validation Metrics (Trial 1, Epoch 4): {'mean_ap': 0.9439633020934596, 'mean_precision': 0.8824050422791313, 'mean_recall': 0.9098238280083607, 'mean_f1': 0.8920493432167201, 'label_aps': {'bridges_any': 0.8342723712466737, 'buildings_any': 0.9926156637739627, 'buildings_affected_or_greater': 0.8780435926382222, 'buildings_minor_or_greater': 0.7527124715066782, 'debris_any': 0.7804759117358534, 'flooding_any': 0.8666195262157244, 'flooding_structures': 0.8462178705570731, 'roads_any': 0.9874138948489448, 'roads_damage': 0.477330926231008, 'trees_any': 0.9966841771317309, 'trees_damage': 0.8467721648420558, 'water_any': 0.9681891005736539}, 'label_f1s': {'bridges_any': 0.7912087912087912, 'buildings_any': 0.9642184557438794, 'buildings_affected_or_greater': 0.7849462365591398, 'buildings_minor_or_greater': 0.6542056074766355, 'debris_any': 0.73224043715847, 'flooding_any': 0.7861271676300579, 'flooding_structures': 0.8037383177570093, 'roads_any': 0.9385382059800664, 'roads_damage': 0.2

Training: 0it [00:00, ?it/s]<ipython-input-7-0abc238c160f>:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 502it [25:11,  3.01s/it]
Validation: 56it [03:00,  3.22s/it]
[I 2024-12-05 21:22:44,949] Trial 1 finished with value: 0.9447671599425089 and parameters: {'lr': 5.895255252186952e-06, 'dropout_rate': 0.23042276808728862, 'batch_size': 16}. Best is trial 0 with value: 0.9461098084486654.


Validation Metrics (Trial 1, Epoch 5): {'mean_ap': 0.9428816703327637, 'mean_precision': 0.9045614029679381, 'mean_recall': 0.8883248730964467, 'mean_f1': 0.8889461676947279, 'label_aps': {'bridges_any': 0.819940124125014, 'buildings_any': 0.989742920070334, 'buildings_affected_or_greater': 0.86634560141319, 'buildings_minor_or_greater': 0.7694528424883759, 'debris_any': 0.7742338845621968, 'flooding_any': 0.8698350543044159, 'flooding_structures': 0.8572596373529813, 'roads_any': 0.9902560797852431, 'roads_damage': 0.5173924466689943, 'trees_any': 0.9956682593716479, 'trees_damage': 0.8355532871327611, 'water_any': 0.9661332693464975}, 'label_f1s': {'bridges_any': 0.7294117647058823, 'buildings_any': 0.9611374407582939, 'buildings_affected_or_greater': 0.7513812154696132, 'buildings_minor_or_greater': 0.6851851851851852, 'debris_any': 0.75, 'flooding_any': 0.7707006369426752, 'flooding_structures': 0.74, 'roads_any': 0.9431034482758621, 'roads_damage': 0.12, 'trees_any': 0.97564022485

In [ ]:
# Final Training with Best Hyperparameters
# best_params = study.best_params
# Manually set best parameters
best_params = {
    'lr': 7.243499683207418e-06,
    'dropout_rate': 0.1535643673944425,
    'batch_size': 64
}
final_model = CustomDINOv2WithDropout(base_model.base_model, num_labels=len(label_keys), dropout_rate=best_params["dropout_rate"]).to(device)
optimizer = torch.optim.AdamW(final_model.parameters(), lr=best_params["lr"], weight_decay=1e-4)

# Path to save the best model for final training
final_model_path = "/content/final_best_model.pth"

best_val_map = 0.0
patience, no_improvement = 3, 0
accumulation_steps = max(1, 64 // best_params["batch_size"])  # Simulate effective batch size of 64

for epoch in range(15):
    print(f"Final Training, Epoch {epoch + 1}/15")

    # Training
    final_model.train()
    train_loss = 0.0
    train_dataset = StreamDataset(ds, "train", label_keys, image_transforms)
    train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], collate_fn=lambda x: tuple(zip(*x)))

    optimizer.zero_grad()
    for i, (batch_images, batch_labels) in enumerate(tqdm(train_loader, desc="Training")):
        batch_images, batch_labels = torch.stack(batch_images).to(device), torch.stack(batch_labels).to(device)

        with torch.cuda.amp.autocast():
            logits = final_model(batch_images)
            loss = nn.BCEWithLogitsLoss()(logits, batch_labels)
            loss = loss / accumulation_steps  # Normalize for gradient accumulation

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        train_loss += loss.item() * accumulation_steps  # De-normalize for logging

    # Validation
    val_loss, val_logits, val_labels = process_dataset(
        final_model, ds, "validation", label_keys, image_transforms, batch_size=best_params["batch_size"]
    )
    val_metrics = multi_label_metrics(val_logits, val_labels.numpy(), label_keys)
    print(f"Validation Metrics: {val_metrics}")

    # Save the best model during final training
    if val_metrics["mean_ap"] > best_val_map:
        best_val_map = val_metrics["mean_ap"]
        no_improvement = 0
        torch.save(final_model.state_dict(), final_model_path)
        print(f"New best model saved with mAP: {best_val_map:.4f}")
    else:
        no_improvement += 1

    # Early stopping
    if no_improvement >= patience:
        print("Early stopping triggered during final training.")
        break

    torch.cuda.empty_cache()  # Clear memory to prevent fragmentation





Final Training, Epoch 1/15


Training: 0it [00:00, ?it/s]<ipython-input-8-a45078a74e44>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 126it [25:16, 12.03s/it]
Validation: 8it [01:59, 14.82s/it]'(ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs-us-1.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 72dfea20-35ee-490f-8351-58b3aa6eaafe)')' thrown while requesting GET https://huggingface.co/datasets/MITLL/LADI-v2-dataset/resolve/5f2dbfe8c466d32edafd1bab847ec5252309acdb/data/validation/data-00025-of-00040.arrow
Retrying in 1s [Retry 1/5].
Validation: 14it [03:37, 15.53s/it]


Validation Metrics: {'mean_ap': 0.9310317999815452, 'mean_precision': 0.9222227458364122, 'mean_recall': 0.8181546730367274, 'mean_f1': 0.8444613237279097, 'label_aps': {'bridges_any': 0.772843638506266, 'buildings_any': 0.9919956946001534, 'buildings_affected_or_greater': 0.8112341181192707, 'buildings_minor_or_greater': 0.7151842344671382, 'debris_any': 0.7637105409883608, 'flooding_any': 0.8472072540069087, 'flooding_structures': 0.8347202445581658, 'roads_any': 0.9867970574677236, 'roads_damage': 0.3318138258402927, 'trees_any': 0.9954129750060572, 'trees_damage': 0.8058696436384956, 'water_any': 0.9635515373780983}, 'label_f1s': {'bridges_any': 0.5373134328358209, 'buildings_any': 0.9585687382297552, 'buildings_affected_or_greater': 0.6219081272084805, 'buildings_minor_or_greater': 0.5625, 'debris_any': 0.6621621621621622, 'flooding_any': 0.6533864541832669, 'flooding_structures': 0.35294117647058826, 'roads_any': 0.9343308395677473, 'roads_damage': 0.0, 'trees_any': 0.97313797313

Training: 0it [00:00, ?it/s]<ipython-input-8-a45078a74e44>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 126it [24:59, 11.90s/it]
Validation: 14it [03:07, 13.39s/it]


Validation Metrics: {'mean_ap': 0.9423650867933403, 'mean_precision': 0.9380473292425361, 'mean_recall': 0.8175574798447298, 'mean_f1': 0.8548390817039973, 'label_aps': {'bridges_any': 0.8079305298252851, 'buildings_any': 0.9936194504266922, 'buildings_affected_or_greater': 0.8532150141654973, 'buildings_minor_or_greater': 0.7670894160950684, 'debris_any': 0.7851277872386315, 'flooding_any': 0.88437043779595, 'flooding_structures': 0.8647716017376099, 'roads_any': 0.9902096033857772, 'roads_damage': 0.3991635752930742, 'trees_any': 0.9970866040555326, 'trees_damage': 0.8328968172402949, 'water_any': 0.9672304967527258}, 'label_f1s': {'bridges_any': 0.6923076923076923, 'buildings_any': 0.9633113828786454, 'buildings_affected_or_greater': 0.5533596837944664, 'buildings_minor_or_greater': 0.5333333333333333, 'debris_any': 0.6713286713286714, 'flooding_any': 0.7293233082706767, 'flooding_structures': 0.55, 'roads_any': 0.9471919530595139, 'roads_damage': 0.04081632653061224, 'trees_any': 0

...

Validation Metrics: {'mean_ap': 0.9448811915028206, 'mean_precision': 0.8878501825721049, 'mean_recall': 0.923260674828307, 'mean_f1': 0.9018363808829445, 'label_aps': {'bridges_any': 0.8548660733329855, 'buildings_any': 0.9909169160915078, 'buildings_affected_or_greater': 0.8992919000300477, 'buildings_minor_or_greater': 0.7641878387074408, 'debris_any': 0.7652019081740548, 'flooding_any': 0.8931566858349234, 'flooding_structures': 0.9044120387022913, 'roads_any': 0.9890762377893575, 'roads_damage': 0.46053419265697526, 'trees_any': 0.9950188790248704, 'trees_damage': 0.8202010437647077, 'water_any': 0.9669786457503465}, 'label_f1s': {'bridges_any': 0.7142857142857143, 'buildings_any': 0.9690140845070423, 'buildings_affected_or_greater': 0.8184438040345822, 'buildings_minor_or_greater': 0.62, 'debris_any': 0.75, 'flooding_any': 0.8103448275862069, 'flooding_structures': 0.8703703703703703, 'roads_any': 0.9405052974735126, 'roads_damage': 0.4166666666666667, 'trees_any': 0.982153846153

Training: 0it [00:00, ?it/s]<ipython-input-8-a45078a74e44>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 126it [25:31, 12.15s/it]
Validation: 14it [03:29, 14.95s/it]


Validation Metrics: {'mean_ap': 0.9438831386005319, 'mean_precision': 0.890344535848109, 'mean_recall': 0.9089280382203643, 'mean_f1': 0.8958749526903993, 'label_aps': {'bridges_any': 0.8174687482861946, 'buildings_any': 0.9842003007841924, 'buildings_affected_or_greater': 0.8847486651034834, 'buildings_minor_or_greater': 0.7723879949961351, 'debris_any': 0.7686787067493558, 'flooding_any': 0.8884252650953833, 'flooding_structures': 0.8979922751434377, 'roads_any': 0.9900542991958773, 'roads_damage': 0.5009532675382895, 'trees_any': 0.9948776817504971, 'trees_damage': 0.8258053568345683, 'water_any': 0.969220481288845}, 'label_f1s': {'bridges_any': 0.7130434782608696, 'buildings_any': 0.967741935483871, 'buildings_affected_or_greater': 0.7654320987654321, 'buildings_minor_or_greater': 0.66, 'debris_any': 0.7272727272727273, 'flooding_any': 0.7800586510263929, 'flooding_structures': 0.8380952380952381, 'roads_any': 0.9432098765432099, 'roads_damage': 0.4383561643835616, 'trees_any': 0.9

Training: 0it [00:00, ?it/s]<ipython-input-8-a45078a74e44>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 126it [25:22, 12.08s/it]
Validation: 14it [03:08, 13.48s/it]

Validation Metrics: {'mean_ap': 0.9450591180584627, 'mean_precision': 0.9020170771533885, 'mean_recall': 0.9020603165123917, 'mean_f1': 0.8998931978925744, 'label_aps': {'bridges_any': 0.8707753685493205, 'buildings_any': 0.983355809386273, 'buildings_affected_or_greater': 0.8884763562213003, 'buildings_minor_or_greater': 0.7695566513929748, 'debris_any': 0.7642905467741997, 'flooding_any': 0.8903583035532204, 'flooding_structures': 0.9145644863564291, 'roads_any': 0.9871233698219407, 'roads_damage': 0.5209478527875925, 'trees_any': 0.9962796627208002, 'trees_damage': 0.8269515480186858, 'water_any': 0.9688784245869783}, 'label_f1s': {'bridges_any': 0.7884615384615384, 'buildings_any': 0.9631728045325779, 'buildings_affected_or_greater': 0.774390243902439, 'buildings_minor_or_greater': 0.6355140186915887, 'debris_any': 0.7262569832402235, 'flooding_any': 0.7988980716253443, 'flooding_structures': 0.8256880733944955, 'roads_any': 0.9491525423728814, 'roads_damage': 0.4444444444444444, '

In [ ]:
# Evaluate on test set
test_loss, test_logits, test_labels = process_dataset(
    final_model, ds, "test", label_keys, image_transforms, batch_size=best_params["batch_size"]
)


Validation: 17it [03:59, 14.08s/it]


In [ ]:
test_metrics = multi_label_metrics(test_logits, test_labels.numpy(), label_keys)
print(f"Test Metrics: {test_metrics}")

Test Metrics: {'mean_ap': 0.9358319232425576, 'mean_precision': 0.8943476012954142, 'mean_recall': 0.8860208816705336, 'mean_f1': 0.8873895620341293, 'label_aps': {'bridges_any': 0.7043559862041444, 'buildings_any': 0.9842972066618617, 'buildings_affected_or_greater': 0.7072538123012451, 'buildings_minor_or_greater': 0.725720862696994, 'debris_any': 0.6954187820375122, 'flooding_any': 0.6206667841562463, 'flooding_structures': 0.3521433648082837, 'roads_any': 0.9688433380835724, 'roads_damage': 0.1193418948805612, 'trees_any': 0.9957081619718413, 'trees_damage': 0.5814604099602392, 'water_any': 0.9196310651592996}, 'label_f1s': {'bridges_any': 0.6119402985074627, 'buildings_any': 0.9494418910045962, 'buildings_affected_or_greater': 0.6136363636363636, 'buildings_minor_or_greater': 0.5454545454545454, 'debris_any': 0.6153846153846154, 'flooding_any': 0.5454545454545454, 'flooding_structures': 0.47058823529411764, 'roads_any': 0.9155609167671894, 'roads_damage': 0.11764705882352941, 'tre

In [ ]:
# Display label-specific mAP
print("Label-wise mAP:")
for label, ap in test_metrics["label_aps"].items():
    print(f"{label}: {ap:.4f}")

Label-wise mAP:
bridges_any: 0.7044
buildings_any: 0.9843
buildings_affected_or_greater: 0.7073
buildings_minor_or_greater: 0.7257
debris_any: 0.6954
flooding_any: 0.6207
flooding_structures: 0.3521
roads_any: 0.9688
roads_damage: 0.1193
trees_any: 0.9957
trees_damage: 0.5815
water_any: 0.9196


In [ ]:
# Display label-specific F1 scores
print("\nLabel-wise F1 Scores:")
for label, f1 in test_metrics["label_f1s"].items():
    print(f"{label}: {f1:.4f}")


Label-wise F1 Scores:
bridges_any: 0.6119
buildings_any: 0.9494
buildings_affected_or_greater: 0.6136
buildings_minor_or_greater: 0.5455
debris_any: 0.6154
flooding_any: 0.5455
flooding_structures: 0.4706
roads_any: 0.9156
roads_damage: 0.1176
trees_any: 0.9657
trees_damage: 0.5363
water_any: 0.8379
